In [2]:
import pytz
import os
from datetime import datetime, timedelta
import pandas as pd
from pandas.tseries.offsets import MonthBegin
import urllib2
import cPickle
from sunpy.time import TimeRange
from sunpy.instr import goes
from sunpy.net import hek
client = hek.HEKClient()

https://www.ngdc.noaa.gov/stp/solar/solarflares.html

In [3]:
# date_values = pd.date_range(pd.Timestamp('2005-01-01').tz_localize('utc'), 
#                           datetime.now(pytz.utc) - MonthBegin(n=1), freq='MS')
# # date_values = pd.date_range(pd.Timestamp('2014-12-01').tz_localize('utc'), 
# #                           pd.Timestamp('2015-01-01').tz_localize('utc'),  freq='MS')
# # date_values = pd.date_range(pd.Timestamp('2015-01-01').tz_localize('utc'), 
# #                           pd.Timestamp('2015-02-01').tz_localize('utc'),  freq='MS')

# Let's try to move from the most recent to the oldest, just in case it stops at some date kung kelan wala nang data.

In [4]:
# date_values = pd.date_range(pd.Timestamp('1975-01-01').tz_localize('utc'), 
#                           datetime.now(pytz.utc) - MonthBegin(n=1), freq='MS')
date_values = pd.date_range(pd.Timestamp('1975-01-01').tz_localize('utc'), 
                          pd.Timestamp('2005-01-01').tz_localize('utc'),  freq='MS')
date_values = date_values.sort_values(ascending=False)
date_values

DatetimeIndex(['2005-01-01', '2004-12-01', '2004-11-01', '2004-10-01',
               '2004-09-01', '2004-08-01', '2004-07-01', '2004-06-01',
               '2004-05-01', '2004-04-01',
               ...
               '1975-10-01', '1975-09-01', '1975-08-01', '1975-07-01',
               '1975-06-01', '1975-05-01', '1975-04-01', '1975-03-01',
               '1975-02-01', '1975-01-01'],
              dtype='datetime64[ns, UTC]', length=361, freq='-1MS')

# Get from Flares from GOES and AR from SDO

In [5]:
# main_folder_name = 'FL-GOES'
# directory = 'F:\Research/Data/AR-FL/'#.format(main_folder_name)
directory = 'AR-FL/'
if not os.path.exists(directory):
    os.makedirs(directory)

In [ ]:
def get_goes_data(time_range):
    success_data = False
    while not success_data:
        try:
            goes_data = goes.get_goes_event_list(time_range)
            success_data = True
        except urllib2.HTTPError as err:
            pass
    return goes_data

def query_hek_goes(tstart, tend):
    import urllib2
    success_data = False
    while not success_data:
        try:
            result_data = client.query(hek.attrs.Time(tstart,tend),
                                       hek.attrs.EventType('FL'),
                                       hek.attrs.OBS.Observatory == 'GOES')
            success_data = True
        except (urllib2.HTTPError, urllib2.URLError):
            pass
    return result_data

def query_hek(tstart, tend, event_type):
    import urllib2
    success_data = False
    while not success_data:
        try:
            result_data = client.query(hek.attrs.Time(tstart,tend),hek.attrs.EventType(event_type))
            success_data = True
        except (urllib2.HTTPError, urllib2.URLError):
            pass
    return result_data

In [ ]:
%%time
for d in xrange(len(date_values) - 1):
    # Get FL data from GOES
    goes_data_query = query_hek_goes(date_values[d], date_values[d] + MonthBegin(n=1))
    goes_data_query = pd.DataFrame.from_dict(goes_data_query)
    if len(goes_data_query) > 0:
        goes_data = goes_data_query[[
                'ar_noaanum',
                'event_starttime',
                'event_peaktime',
                'event_endtime',
                'fl_goescls',
                'obs_instrument',
                'obs_title',
                'event_coord1',
                'event_coord2',
                'event_coord3',
                'event_coordsys',
                'event_coordunit',
                'event_description',
                'boundbox_c1ll',
                'boundbox_c1ur',
                'hrc_bbox',
                'hrc_boundcc',
                'hrc_coord',
                'hgc_bbox',
                'hgc_boundcc',
                'hgc_coord',
                'hgc_x',
                'hgc_y',
                'hgs_bbox',
                'hgs_boundcc',
                'hgs_coord',
                'hgs_x',
                'hgs_y',
                'hpc_bbox',
                'hpc_boundcc',
                'hpc_coord',
                'hpc_geom',
                'hpc_radius',
                'hpc_x',
                'hpc_y',
                'hrc_a',
                'hrc_bbox',
                'hrc_boundcc',
                'hrc_coord',
                'hrc_r',
                'fl_efoldtime',
                'fl_efoldtimeunit',
                'fl_fluence',
                'fl_fluenceunit',
                'fl_halphaclass',
                'fl_peakem',
                'fl_peakemunit',
                'fl_peakflux',
                'fl_peakfluxunit',
                'fl_peaktemp',
                'fl_peaktempunit'
            ]]
        goes_data.fl_goescls = goes_data.fl_goescls.apply(lambda x: str(x))
        goes_data['flux'] = goes_data.fl_goescls.apply(lambda x: goes.flareclass_to_flux(x))
        goes_data.to_csv('{0}GOES-FL-{1}.csv'.format(directory, str(date_values[d].to_period('M'))))
    print "Done FL, {0}".format(str(date_values[d].to_period('M')))

/home/norman/anaconda2/lib/python2.7/site-packages/pandas/core/generic.py:2387: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/home/norman/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Done FL, 2005-01
Done FL, 2004-12
Done FL, 2004-11
Done FL, 2004-10
Done FL, 2004-09
Done FL, 2004-08
Done FL, 2004-07
Done FL, 2004-06
Done FL, 2004-05
Done FL, 2004-04
Done FL, 2004-03
Done FL, 2004-02
Done FL, 2004-01
Done FL, 2003-12
Done FL, 2003-11
Done FL, 2003-10
Done FL, 2003-09
Done FL, 2003-08
Done FL, 2003-07
Done FL, 2003-06
Done FL, 2003-05
Done FL, 2003-04
Done FL, 2003-03
Done FL, 2003-02

In [23]:
for i in goes_data_query.columns: print i

SOL_standard
active
ar_compactnesscls
ar_mcintoshcls
ar_mtwilsoncls
ar_noaaclass
ar_noaanum
ar_numspots
ar_penumbracls
ar_polarity
ar_zurichcls
area_atdiskcenter
area_atdiskcenteruncert
area_raw
area_uncert
area_unit
bound_ccnsteps
bound_ccstartc1
bound_ccstartc2
bound_chaincode
boundbox_c1ll
boundbox_c1ur
boundbox_c2ll
boundbox_c2ur
chaincodetype
comment_count
concept
event_avg_rating
event_c1error
event_c2error
event_clippedspatial
event_clippedtemporal
event_coord1
event_coord2
event_coord3
event_coordsys
event_coordunit
event_description
event_endtime
event_expires
event_importance
event_importance_num_ratings
event_mapurl
event_maskurl
event_npixels
event_peaktime
event_pixelunit
event_probability
event_score
event_starttime
event_testflag
event_title
event_type
eventtype
fl_efoldtime
fl_efoldtimeunit
fl_fluence
fl_fluenceunit
fl_goescls
fl_halphaclass
fl_peakem
fl_peakemunit
fl_peakflux
fl_peakfluxunit
fl_peaktemp
fl_peaktempunit
frm_contact
frm_daterun
frm_humanflag
frm_identifi

# No data for 2009-08??
May months talaga na walang data. hmm.

In [13]:
goes_data_query

""


In [16]:
%%time
for d in xrange(len(date_values) - 1):
    # Get FL data
    AR_data = query_hek(date_values[d], date_values[d+1], 'AR')
    AR_data = pd.DataFrame.from_dict(AR_data)
    AR_data_selected = AR_data[[
            'ar_noaanum',
            'ar_mcintoshcls', 
            'search_frm_name',
            'search_instrument',
            'search_observatory',
            'sharp_noaa_ars',
            'obs_instrument',
            'obs_lastprocessingdate',
            'obs_levelnum',
            'obs_meanwavel',
            'obs_observatory',
            'obs_title',
            'obs_wavelunit',
            'event_coord1',
            'event_coord2',
            'boundbox_c1ll',
            'boundbox_c1ur',
            'hrc_bbox',
            'hrc_boundcc',
            'hrc_coord',
            'hgc_bbox',
            'hgc_boundcc',
            'hgc_coord',
            'hgc_x',
            'hgc_y',
            'hgs_bbox',
            'hgs_boundcc',
            'hgs_coord',
            'hgs_x',
            'hgs_y',
            'hpc_bbox',
            'hpc_boundcc',
            'hpc_coord',
            'hpc_geom',
            'hpc_radius',
            'hpc_x',
            'hpc_y',
            'hrc_a',
            'hrc_bbox',
            'hrc_boundcc',
            'hrc_coord',
            'hrc_r',
        ]]
    AR_data_selected = AR_data_selected.dropna(axis=0, subset=['ar_noaanum'])
    AR_data_selected = AR_data_selected.reset_index().drop(['index'], 1)
    AR_data_selected.to_csv('{0}SDO-AR-{1}.csv'.format(directory, str(date_values[d].to_period('M'))))
    print "Done AR, {0}".format(str(date_values[d].to_period('M')))

Done AR, 2014-12
CPU times: user 1.01 s, sys: 192 ms, total: 1.2 s
Wall time: 2min 29s


In [6]:
%%time
goes_data = pd.DataFrame.from_dict(get_goes_data(time_range))

Wall time: 8.75 s


In [6]:
goes_data[:3]

,end_time,event_date,goes_class,goes_location,noaa_active_region,peak_time,start_time
0,2015-01-01 00:17:00,2014-12-31,C1.2,"(0, 0)",12253,2014-12-31 23:39:00,2014-12-31 23:02:00
1,2015-01-01 05:17:00,2015-01-01,C2.1,"(-50, -7)",12253,2015-01-01 05:07:00,2015-01-01 04:57:00
2,2015-01-01 07:38:00,2015-01-01,C2.1,"(-49, -6)",12253,2015-01-01 07:30:00,2015-01-01 07:22:00


In [7]:
goes_data['flux'] = goes_data.goes_class.apply(lambda x: goes.flareclass_to_flux(x))

AttributeError: 'module' object has no attribute 'flareclass_to_flux'

In [24]:
(goes_data.loc[0, 'flux'])

<Quantity 6.2e-07 W / m2>

In [25]:
goes_data.noaa_active_region.value_counts()

12480    35
12488    17
0        16
12489    13
12487     8
12484     7
12473     6
12483     5
12490     4
12485     3
12476     3
12481     2
12482     1
12479     1
12478     1
12477     1
Name: noaa_active_region, dtype: int64

#Get AR data

C:\Users\Akared\Anaconda\lib\site-packages\pandas\computation\__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [12]:
%%time
tstart = '2016/01/01 00:00'
tend = '2016/02/01 00:00'


Wall time: 4min 5s


In [13]:

AR_data[:3]

,SOL_standard,absnetcurrenthelicity,active,ar_axislength,ar_compactnesscls,ar_lengthunit,ar_mcintoshcls,ar_mtwilsoncls,ar_neutrallength,ar_noaaclass,...,sharp_noaa_ars,sum_overlap_scores,totalenergydensityunit,totalphotoenergy,totalphotoenergydensity,totalphotoenergyunit,twistunit,unsignedcurrenthelicity,unsignedflux,unsignedvertcurrent
0,SOL2015-12-31T20:00:00L326C085,7.069000,true,None,,,,,None,,...,12472,0.968438855073890936,ergs/cm,1.643104e+30,None,ergs,1/mm,681.268005,17775.169922,1.619847e+13
1,SOL2015-12-31T20:00:00L332C111,203.585007,true,None,,,,,None,,...,12473,1.07421527314572618,ergs/cm,9.675007e+30,None,ergs,1/mm,1672.811035,33016.328125,3.264197e+13
2,SOL2015-12-31T20:00:00L241C079,1.139000,true,None,,,,,None,,...,,0,ergs/cm,3.251704e+28,None,ergs,1/mm,18.312000,254.977600,3.841124e+11


In [15]:
for i in AR_data.columns:
    print i

SOL_standard
absnetcurrenthelicity
active
ar_axislength
ar_compactnesscls
ar_lengthunit
ar_mcintoshcls
ar_mtwilsoncls
ar_neutrallength
ar_noaaclass
ar_noaanum
ar_numspots
ar_penumbracls
ar_pilcurvature
ar_polarity
ar_spotarearaw
ar_spotarearawuncert
ar_spotarearawunit
ar_spotarearepr
ar_spotarearepruncert
ar_spotareareprunit
ar_sumnegsignedflux
ar_sumpossignedflux
ar_zurichcls
area_atdiskcenter
area_atdiskcenteruncert
area_raw
area_uncert
area_unit
bound_ccnsteps
bound_ccstartc1
bound_ccstartc2
bound_chaincode
boundbox_c1ll
boundbox_c1ur
boundbox_c2ll
boundbox_c2ur
chaincodetype
comment_count
concept
currentdensityunit
currenthelicityunit
currentunit
event_avg_rating
event_c1error
event_c2error
event_clippedspatial
event_clippedtemporal
event_coord1
event_coord2
event_coord3
event_coordsys
event_coordunit
event_description
event_endtime
event_expires
event_importance
event_importance_num_ratings
event_mapurl
event_maskurl
event_npixels
event_pixelunit
event_probability
event_score
even

In [27]:
AR_data_selected = AR_data[[
        'ar_noaanum',
        'ar_mcintoshcls', 
        'search_frm_name',
        'search_instrument',
        'search_observatory',
        'sharp_noaa_ars',
        'obs_instrument',
        'obs_lastprocessingdate',
        'obs_levelnum',
        'obs_meanwavel',
        'obs_observatory',
        'obs_title',
        'obs_wavelunit',
        'event_coord1',
        'event_coord2',
        'boundbox_c1ll',
        'boundbox_c1ur',
        'hrc_bbox',
        'hrc_boundcc',
        'hrc_coord',
        'hgc_bbox',
        'hgc_boundcc',
        'hgc_coord',
        'hgc_x',
        'hgc_y',
        'hgs_bbox',
        'hgs_boundcc',
        'hgs_coord',
        'hgs_x',
        'hgs_y',
        'hpc_bbox',
        'hpc_boundcc',
        'hpc_coord',
        'hpc_geom',
        'hpc_radius',
        'hpc_x',
        'hpc_y',
        'hrc_a',
        'hrc_bbox',
        'hrc_boundcc',
        'hrc_coord',
        'hrc_r',
    ]]
AR_data_selected = AR_data_selected.dropna(axis=0, subset=['ar_noaanum'])
AR_data_selected[:3]

,ar_noaanum,ar_mcintoshcls,search_frm_name,search_instrument,search_observatory,sharp_noaa_ars,obs_instrument,obs_lastprocessingdate,obs_levelnum,obs_meanwavel,...,hpc_coord,hpc_geom,hpc_radius,hpc_x,hpc_y,hrc_a,hrc_bbox,hrc_boundcc,hrc_coord,hrc_r
0,12472.0,,HMI SHARP,HMI,SDO,12472,HMI,,NaN,0.00005,...,POINT(730.93032 126.802698),0103000000010000000500000071C971A7F4538340BC57...,741.847731623802702,730.93032,126.802698,279.841791,"POLYGON((0.634395 271.432321,0.903503 269.8930...",,POINT(0.760681474682731 279.841790983815),0.760681
1,12473.0,,HMI SHARP,HMI,SDO,12473,HMI,,NaN,0.00005,...,POINT(741.85422 -331.472448),01030000000100000005000000D0D6C1C11EC781403FC7...,812.540255935003529,741.85422,-331.472448,245.924137,"POLYGON((0.727536 233.301616,0.932699 241.1062...",,POINT(0.833168713437757 245.924136782866),0.833169
8,12472.0,,HMI SHARP,HMI,SDO,12472,HMI,,NaN,0.00005,...,POINT(751.9668 129.13407),0103000000010000000500000071AC8BDBE8348440BF9A...,762.974230454086182,751.96680,129.134070,279.744267,"POLYGON((0.665023 274.438474,0.916235 272.1049...",,POINT(0.782342969161661 279.744267271899),0.782343


In [30]:
AR_data_selected.reset_index().drop(['index'], 1)[:3]

,ar_noaanum,ar_mcintoshcls,search_frm_name,search_instrument,search_observatory,sharp_noaa_ars,obs_instrument,obs_lastprocessingdate,obs_levelnum,obs_meanwavel,...,hpc_coord,hpc_geom,hpc_radius,hpc_x,hpc_y,hrc_a,hrc_bbox,hrc_boundcc,hrc_coord,hrc_r
0,12472.0,,HMI SHARP,HMI,SDO,12472,HMI,,NaN,0.00005,...,POINT(730.93032 126.802698),0103000000010000000500000071C971A7F4538340BC57...,741.847731623802702,730.93032,126.802698,279.841791,"POLYGON((0.634395 271.432321,0.903503 269.8930...",,POINT(0.760681474682731 279.841790983815),0.760681
1,12473.0,,HMI SHARP,HMI,SDO,12473,HMI,,NaN,0.00005,...,POINT(741.85422 -331.472448),01030000000100000005000000D0D6C1C11EC781403FC7...,812.540255935003529,741.85422,-331.472448,245.924137,"POLYGON((0.727536 233.301616,0.932699 241.1062...",,POINT(0.833168713437757 245.924136782866),0.833169
2,12472.0,,HMI SHARP,HMI,SDO,12472,HMI,,NaN,0.00005,...,POINT(751.9668 129.13407),0103000000010000000500000071AC8BDBE8348440BF9A...,762.974230454086182,751.96680,129.134070,279.744267,"POLYGON((0.665023 274.438474,0.916235 272.1049...",,POINT(0.782342969161661 279.744267271899),0.782343


In [18]:
set(AR_data_selected.search_observatory)

{u'SDO', u'nil'}